# Tutorial for the OpenFermion-Dirac interface

The goal of this interface is to perform relativistic or non-relativistic calculation of 
chemical compounds through the Dirac program (see http://diracprogram.org)
in OpenFermion (https://github.com/quantumlib/OpenFermion). 
The resulting molecular Hamiltonian can then be encoded into a qubit Hamiltonian thanks to the Jordan-Wigner
transformation, for instance.

## Non relativistic calculation

First, define your atom or molecule. Here, the example of the hydrogen molecule is provided.

In [1]:
bond_length = 1.0
multiplicity = 1
charge = 0
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length))]

Then, specify the basis set. For practical reason, the small STO-3G basis set is considered here. More complex basis sets can be used to perform Hartree-Fock (HF) or Coupled Cluster Single and Doubles (CCSD), but it becomes rapidly untractable for Full Configuration Interaction (FCI).

In [2]:
basis = 'STO-3G'

To know more about the valid basis sets, we refer the reader to the DIRAC documentation (http://diracprogram.org).

Given that the filename does not contain information about the bond length or the type of calculation, it is interesting to add it manually. We will perform here a CCSD calculation.

In [3]:
run_ccsd = True
if run_ccsd:
    description = 'R' + str(bond_length) + '_ccsd'
else:
    description = 'R' + str(bond_length) + '_scf'

Now that every parameters are defined, one can construct the molecule thanks to the ```MolecularData_Dirac``` command

In [4]:
import os
data_directory=os.getcwd() + "/" + "work_dir"

from openfermion_dirac import MolecularData_Dirac

molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               description=description,
                               data_directory=data_directory)

Time to start the calculation ! First, defines which type of calculation you want to perform. The choices are SCF (Hartree-Fock) or CCSD. If you don't set any option, it will run an SCF calculation automatically. 
If you set ```run_ccsd = True```, it will perform a CCSD calculation. Performing CCSD also gives you access to the second-order Moller-Plesset perturbation (MP2) and the Hartree-Fock (HF) energy.

In [5]:
run_ccsd = True

Several files are generated during the calculation: 
- The input file (.inp)
- The geometry file (.xyz)
- The output file (.out)
- The integral files (MRCONEE and MDCINT)
- The FCIDUMP file
- Optionally, an hdf5 file which saves several informations (as shown latter on).

You can define all the files that you want to keep or to discard at the end of the calculation. Note that the geometry of the molecule and the input file are included in the Dirac output file, such that only this file can be kept.
The MRCONEE and MDCINT contain the one- and two-electron integrals, respectively. The ```dirac_openfermion_mointegral_export.x``` transforms those files into a FCIDUMP file containing all the molecular integrals. Only the latter file is then needed.

In [6]:
delete_input = True
delete_xyz = True
delete_output = False
delete_MRCONEE = True
delete_MDCINT = True
delete_FCIDUMP = False

To start the calculation, use the command ```run_dirac``` as follows:

In [7]:
from openfermion_dirac import run_dirac

molecule = run_dirac(molecule,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    delete_FCIDUMP=delete_FCIDUMP,
                    run_ccsd=run_ccsd)

Starting Dirac calculation


Creation of the FCIDUMP file



Now, one can extract the second quantized Hamiltonian from the object ```molecule```

In [8]:
molecular_hamiltonian = molecule.get_molecular_hamiltonian()[0]
print(molecular_hamiltonian)

() 0.5291772083
((0, 1), (0, 0)) -1.11084417705
((1, 1), (1, 0)) -1.11084417705
((2, 1), (2, 0)) -0.5891210130952
((3, 1), (3, 0)) -0.5891210130952
((0, 1), (0, 1), (0, 0), (0, 0)) 0.313201246336
((0, 1), (0, 1), (2, 0), (2, 0)) 0.0983952893027
((0, 1), (1, 1), (1, 0), (0, 0)) 0.313201246336
((0, 1), (1, 1), (3, 0), (2, 0)) 0.0983952893027
((0, 1), (2, 1), (0, 0), (2, 0)) 0.0983952893027
((0, 1), (2, 1), (2, 0), (0, 0)) 0.3108533766756
((0, 1), (3, 1), (1, 0), (2, 0)) 0.0983952893027
((0, 1), (3, 1), (3, 0), (0, 0)) 0.3108533766756
((1, 1), (0, 1), (0, 0), (1, 0)) 0.313201246336
((1, 1), (0, 1), (2, 0), (3, 0)) 0.0983952893027
((1, 1), (1, 1), (1, 0), (1, 0)) 0.313201246336
((1, 1), (1, 1), (3, 0), (3, 0)) 0.0983952893027
((1, 1), (2, 1), (0, 0), (3, 0)) 0.0983952893027
((1, 1), (2, 1), (2, 0), (1, 0)) 0.3108533766756
((1, 1), (3, 1), (1, 0), (3, 0)) 0.0983952893027
((1, 1), (3, 1), (3, 0), (1, 0)) 0.3108533766756
((2, 1), (0, 1), (0, 0), (2, 0)) 0.3108533766756
((2, 1), (0, 1), (2, 0)

and encode it into the qubit Hamiltonian thanks to the Jordan-Wigner transformation, provided by OpenFermion:

In [9]:
from openfermion.transforms import jordan_wigner

qubit_hamiltonian = jordan_wigner(molecular_hamiltonian)
print(qubit_hamiltonian)

-0.32760821099250037 [] +
-0.04919764465135 [X0 X1 Y2 Y3] +
0.04919764465135 [X0 Y1 Y2 X3] +
0.04919764465135 [Y0 X1 X2 Y3] +
-0.04919764465135 [Y0 Y1 X2 X3] +
0.13716573333275 [Z0] +
0.156600623168 [Z0 Z1] +
0.10622904368644998 [Z0 Z2] +
0.1554266883378 [Z0 Z3] +
0.13716573333275 [Z1] +
0.1554266883378 [Z1 Z2] +
0.10622904368644998 [Z1 Z3] +
-0.13036290911284995 [Z2] +
0.1632676836362 [Z2 Z3] +
-0.13036290911284995 [Z3]


The energies calculated from the Dirac program are extracted as follows:

In [10]:
print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_energies()[0]))
print('MP2 energy of {} Hartree.'.format(molecule.get_energies()[1]))
print('CCSD energy of {} Hartree.'.format(molecule.get_energies()[2]))

Hartree-Fock energy of -1.0661086531285879 Hartree.
MP2 energy of -1.086665367911369 Hartree.
CCSD energy of -1.101150333445638 Hartree.


Note that it is usual to set consider the point nucleus model instead of the Gaussian charge distribution (default)
in a non-relativistic calculation. This can be set by specifying:

In [11]:
point_nucleus = True

molecule = run_dirac(molecule,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    delete_FCIDUMP=delete_FCIDUMP,
                    run_ccsd=run_ccsd,
                    point_nucleus=point_nucleus)

print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_energies()[0]))
print('MP2 energy of {} Hartree.'.format(molecule.get_energies()[1]))
print('CCSD energy of {} Hartree.'.format(molecule.get_energies()[2]))

Starting Dirac calculation


Creation of the FCIDUMP file

Hartree-Fock energy of -1.0661086541045273 Hartree.
MP2 energy of -1.086665368896307 Hartree.
CCSD energy of -1.101150334281559 Hartree.


We can observe small changes in the energies. Then one can build the qubit Hamiltonian, which operates in the Fock space (i.e., independently of the number of particle) and is solved
in OpenFermion with the eigenspectrum function:

In [12]:
from openfermion.utils import eigenspectrum

evs = eigenspectrum(qubit_hamiltonian)
print('Solving the Qubit Hamiltonian (Jordan-Wigner): \n {}'.format(evs))

Solving the Qubit Hamiltonian (Jordan-Wigner): 
 [-1.10115033 -0.74587181 -0.74587181 -0.74587181 -0.60860674 -0.60860674
 -0.58166697 -0.58166697 -0.35229065 -0.06021533 -0.06021533 -0.0599438
 -0.0599438   0.0390476   0.50196591  0.52917721]


## DFT Calculation

Density-functional theory is one of the most widely used approach in quantum chemistry due to its mean-field-like formulation, thus leading to a much cheaper computational cost in comparison to multiconfigurational wavefunction methods. In Dirac program, several functionals are available (see http://www.diracprogram.org/doc/master/manual/dftcfun.html?highlight=dft) and can be called in this interface by specifying run_dirac="X", where X is the choosen approximate exchange-correlation functional.

In [13]:
run_dft="LDA"
description = 'R' + str(bond_length) + '_' + run_dft

molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               description=description,
                               data_directory=data_directory)

molecule = run_dirac(molecule,
                    point_nucleus=point_nucleus,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    delete_FCIDUMP=delete_FCIDUMP,
                    run_dft=run_dft)

print('DFT energy of {} Hartree.'.format(molecule.get_energies()[0]))

Starting Dirac calculation


Creation of the FCIDUMP file

DFT energy of -1.0849003105849406 Hartree.


## Relativistic Calculation

To perform a relativistic calculation by considering the full Dirac-Coulomb Hamiltonian, a simple option has to be switched on. Note that one should not switch on the ```point_nucleus``` option for relativistic calculation.

In [14]:
relativistic = True
basis = "STO-3G"
bond_length = 1.0
multiplicity = 1
charge = 0
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length))]
run_ccsd = True
if run_ccsd:
    description = 'R' + str(bond_length) + '_ccsd'
else:
    description = 'R' + str(bond_length) + '_scf'

molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               relativistic=relativistic,
                               description=description,
                               data_directory=data_directory)

molecule = run_dirac(molecule,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    delete_FCIDUMP=delete_FCIDUMP,
                    run_ccsd=run_ccsd,
                    relativistic=relativistic)

molecular_hamiltonian = molecule.get_molecular_hamiltonian()[0]
qubit_hamiltonian = jordan_wigner(molecular_hamiltonian)
evs = eigenspectrum(qubit_hamiltonian)
print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_energies()[0]))
print('MP2 energy of {} Hartree.'.format(molecule.get_energies()[1]))
print('CCSD energy of {} Hartree.'.format(molecule.get_energies()[2]))
print('Solving the Qubit Hamiltonian (Jordan-Wigner): \n {}'.format(evs))

Starting Dirac calculation


Creation of the FCIDUMP file

Hartree-Fock energy of -1.0661226627178420 Hartree.
MP2 energy of -1.086679591591157 Hartree.
CCSD energy of -1.101165704413968 Hartree.
Solving the Qubit Hamiltonian (Jordan-Wigner): 
 [-1.1011657  -0.74591589 -0.74591589 -0.74591589 -0.60865735 -0.60865735
 -0.58167347 -0.58167347 -0.35233876 -0.06029881 -0.06029881 -0.05998437
 -0.05998437  0.03896802  0.50187888  0.52917721]


Small differences are seen between the relativistic energies and the non-relativistic ones obtained previously.
Of course, the difference is quite small for the hydrogen molecule but it is known to be very important especially for heavy elements.

### Change the speed of light

One can play with the speed of light (137 a.u.) in order to increase the relativistic effects:

In [ ]:
speed_of_light = 3.0

molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               speed_of_light=speed_of_light,
                               description=description,
                               relativistic=relativistic,
                               data_directory=data_directory)

molecule = run_dirac(molecule,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    delete_FCIDUMP=delete_FCIDUMP,
                    run_ccsd=run_ccsd,
                    relativistic=relativistic,
                    speed_of_light=speed_of_light)

molecular_hamiltonian = molecule.get_molecular_hamiltonian()[0]
qubit_hamiltonian = jordan_wigner(molecular_hamiltonian)
evs = eigenspectrum(qubit_hamiltonian)
print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_energies()[0]))
print('MP2 energy of {} Hartree.'.format(molecule.get_energies()[1]))
print('CCSD energy of {} Hartree.'.format(molecule.get_energies()[2]))
print('Solving the Qubit Hamiltonian (Jordan-Wigner): \n {}'.format(evs))

Starting Dirac calculation


Creation of the FCIDUMP file

Hartree-Fock energy of -1.0942741633461521 Hartree.
MP2 energy of -1.115281690754091 Hartree.
CCSD energy of -1.132134955528720 Hartree.
Solving the Qubit Hamiltonian (Jordan-Wigner): 
 [-1.13213496 -0.82925112 -0.82925112 -0.82925112 -0.70678081 -0.70678081
 -0.59452925 -0.59452925 -0.44310418 -0.21989424 -0.21989424 -0.13468642
 -0.13468642 -0.10968398  0.33171782  0.52917721]


The change in energy is now much more important, showing the significance of considering relativistic effects in this case.

# Other more specific options

## Properties

Different properties can be computed within the Dirac program.
One can ask for properties calculation by creating a list ```properties = []``` containing keywords corresponding to the given properties of interest.

In [ ]:
properties = ['MOLGRD','DIPOLE','QUADRUPOLE','EFG','POLARIZABILITY'] 
delete_output = False
molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               description=description,
                               data_directory=data_directory)

molecule = run_dirac(molecule,
                    point_nucleus=point_nucleus,
                    properties=properties,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    delete_FCIDUMP=delete_FCIDUMP,
                    run_ccsd=run_ccsd)

print('Electric dipole moment: {}'.format(molecule.get_elecdipole()))
print('Electric quadrupole moment: {}'.format(molecule.get_elecquadrupole()))
print('Electric (dipole) polarizability: {}'.format(molecule.get_elecpolarizability()))

Starting Dirac calculation


Creation of the FCIDUMP file

Electric dipole moment: [0.0, 0.0, 0.0]
Electric quadrupole moment: [[-0.654525960667, 0.0, 0.0], [0.0, -0.654525960667, 0.0], [0.0, 0.0, 1.309051921334]]
Electric (dipole) polarizability: [-0.0, -0.0, 5.20452436]


For now, the electric dipole, quadrupole and polarizability can be extracted. For other properties, one has to look directly at the output.

## Save option

Let us go back in a non relativistic framework (although everything that follows can be done with the ```relativistic = True``` option).
More options can be set in the calculation. The first one is the ```save``` option that allows you to save your calculation in the HDF5 format.
Note that ```delete_FCIDUMP``` has to be equal to ```False```. The ouput can be deleted, but I advise not to do it as it contains much more information on the performed calculation.

In [ ]:
save = True
point_nucleus = True
properties = ['MOLGRD','DIPOLE','QUADRUPOLE','EFG','POLARIZABILITY']

molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               description=description,
                               data_directory=data_directory)

molecule = run_dirac(molecule,
                    point_nucleus=point_nucleus,
                    properties=properties,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    delete_FCIDUMP=delete_FCIDUMP,
                    run_ccsd=run_ccsd,
                    save=save)

Starting Dirac calculation


Creation of the FCIDUMP file


Saving the results

[0.0, 0.0, 0.0] [[-0.654525960667, 0.0, 0.0], [0.0, -0.654525960667, 0.0], [0.0, 0.0, 1.309051921334]] [-0.0, -0.0, 5.20452436]
hey


Then check is the HDF5 file has been created

In [ ]:
import sys
#if os.path.exists("{}/{}.hdf5".format(data_directory,molecule.name)) is False:
if os.path.exists("{}.hdf5".format(molecule.filename)) is False:
      sys.exit(0)
print("HDF5 file found, loading from file. Name : {}".format(molecule.get_from_file('name')))

HDF5 file found, loading from file. Name : b'H2_STO-3G_singlet_R1.0_ccsd'


You can extract the following information from this file thanks to the function ```get_from_file()```:

In [ ]:
print('Name : {}'.format(molecule.get_from_file('name')))
print('The description is {}.'.format(molecule.get_from_file('description')))
print('System with atoms {} and protons {}.'.format(molecule.get_from_file('atoms'),
                                    molecule.get_from_file('protons')))
print('System of {} atoms and {} electrons.'.format(molecule.get_from_file('n_atoms'),
                                                             molecule.get_from_file('n_electrons')))
print('With charge {} and multiplicity {}.'.format(molecule.get_from_file('charge'),
                                                  molecule.get_from_file('multiplicity')))
print('With basis-set {}.'.format(molecule.get_from_file('basis')))
print('With {} spin-orbitals with corresponding energies {}.'.format(molecule.get_from_file('n_orbitals'),
                                                                     molecule.get_from_file('orbital_energies')))
print('With {} qubits.'.format(molecule.get_from_file('n_qubits')))
print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_from_file('hf_energy')))
print('MP2 energy of {} Hartree.'.format(molecule.get_from_file('mp2_energy')))
print('CCSD energy of {} Hartree.'.format(molecule.get_from_file('ccsd_energy')))

E_core=molecule.get_from_file('nuclear_repulsion')
one_body_coeff = molecule.get_from_file('one_body_coefficients')
two_body_coeff = molecule.get_from_file('two_body_coefficients')
molecular_ham_print = molecule.get_from_file('print_molecular_hamiltonian')
print('Core energy : {} Hartree.'.format(E_core))
print('Electric dipole moment: {}'.format(molecule.get_from_file('elec_dipole')))
print('Electric quadrupole moment: {}'.format(molecule.get_from_file('elec_quadrupole')))
print('Electric (dipole) polarizability: {}'.format(molecule.get_from_file('elec_polarizability')))
print('Molecular Hamiltonian : {}'.format(molecular_ham_print))

Name : b'H2_STO-3G_singlet_R1.0_ccsd'
The description is b'R1.0_ccsd'.
System with atoms [b'H' b'H'] and protons [1 1].
System of 2 atoms and 2 electrons.
With charge 0 and multiplicity 1.
With basis-set b'STO-3G'.
With 4 spin-orbitals with corresponding energies {1: -0.5502018819084, 2: -0.5502018819084, 3: -0.03379116269064, 4: -0.03379116269064}.
With 4 qubits.
Hartree-Fock energy of -1.0661086541045273 Hartree.
MP2 energy of -1.086665368896307 Hartree.
CCSD energy of -1.101150334281559 Hartree.
Core energy : 0.5291772083 Hartree.
Electric dipole moment: [0. 0. 0.]
Electric quadrupole moment: [[-0.65452596  0.          0.        ]
 [ 0.         -0.65452596  0.        ]
 [ 0.          0.          1.30905192]]
Electric (dipole) polarizability: [-0.         -0.          5.20452436]
Molecular Hamiltonian : () 0.5291772083
((0, 1), (0, 0)) -1.110844177538
((1, 1), (1, 0)) -1.110844177538
((2, 1), (2, 0)) -0.5891210139955
((3, 1), (3, 0)) -0.5891210139955
((0, 1), (0, 1), (0, 0), (0, 0)) 

Note that this molecular Hamiltonian cannot be used directly to construct the qubit Hamiltonian. It has to be reconstruct first thanks to the core energy and the one- and two-body coefficients:

In [ ]:
from openfermion.ops import InteractionOperator

molecular_ham = InteractionOperator(float(E_core),one_body_coeff,two_body_coeff)
qubit_ham = jordan_wigner(molecular_ham)
evs = eigenspectrum(qubit_ham)
print("Solution of the qubit Hamiltonian :\n {}".format(evs))

Solution of the qubit Hamiltonian :
 [-1.10115033 -0.74587181 -0.74587181 -0.74587181 -0.60860674 -0.60860674
 -0.58166697 -0.58166697 -0.35229065 -0.06021533 -0.06021533 -0.05994381
 -0.05994381  0.0390476   0.50196591  0.52917721]


## Active space option

Let us take the example of the lithium hydride molecule LiH:

In [ ]:
basis = 'STO-3G'
bond_length = 2.0
multiplicity = 1
charge = 0

delete_input = True
delete_xyz = True
delete_output = False
delete_MRCONEE = True
delete_MDCINT = True
delete_FCIDUMP = False
geometry = [('Li', (0., 0., 0.)), ('H', (0., 0., bond_length))]
description = "_R" + str(bond_length) + "_scf"

molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               description=description,
                               data_directory=data_directory)

The option ```active``` is a list of 3 real numbers:
- first number : lowest energy
- second number : highest energy
- third number : minimum gap required between the lowest (highest) minimum gap required between the neighbor energy of the lowest and highest energy set previously. If the gap is lower than this number, the lowest (or highest) energy is shifted down (or up) until the gap is larger to the third number.

The orbitals with energy in between the first and second number are considered active. Of course, one should choose the active space such that no orbital with energy higher than the second number is occupied, and such that the relevant orbitals are considered active.
The electrons in the core orbitals (with energy below the first number) are frozen, while the orbitals with energy higher than the second number are discarded.

For LiH, let's first do a calculation to have information on the orbital energies. To get the orbitals, we only need the SCF calculation (i.e., switch-off ```run_ccsd```, which is the default).

In [ ]:
save = True
point_nucleus=True

molecule = run_dirac(molecule,
                    point_nucleus=point_nucleus,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    delete_FCIDUMP=delete_FCIDUMP,
                    save=save)

print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_energies()[0]))

# This could be done to check if the saving file exist already or not.
#if os.path.exists("{}/{}.hdf5".format(data_directory,molecule.name)) is False:
if os.path.exists("{}.hdf5".format(molecule.filename)) is False:
      print("No file found. You should first run a calculation with save=True (see LiH_save.py)")
      sys.exit(0)

print("The spin-orbitals and their associated energy:{}\n".format(molecule.get_from_file("orbital_energies")))

Starting Dirac calculation


Creation of the FCIDUMP file


Saving the results

None None None
hey
Hartree-Fock energy of -7.8309055873879219 Hartree.
The spin-orbitals and their associated energy:{1: -2.361187885634, 3: -0.2501066514045, 5: 0.07327904161477, 7: 0.1621056645776, 9: 0.1621056645776, 11: 0.4326451334071, 2: -2.361187885634, 4: -0.2501066514045, 6: 0.07327904161477, 8: 0.1621056645776, 10: 0.1621056645776, 12: 0.4326451334071}



LiH is a system with 4 electrons. We see that the first spatial orbital (indexed by 1 and 2) has an energy much lower than the second one (indexed by 3 and 4). Consequently, one can freeze the two electrons in this orbital and consider only the 2 other electrons as active. We can also get rid of the last spatial orbital with the highest energy. By doing so, only 8 spin-orbitals with 2 electrons will form our active space.

In [ ]:
active = [-2.0,0.3,0.1]

run_ccsd=True

molecule = run_dirac(molecule,
                    point_nucleus=point_nucleus,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    delete_FCIDUMP=delete_FCIDUMP,
                    run_ccsd=run_ccsd,
                    active=active)

molecular_hamiltonian = molecule.get_molecular_hamiltonian()[0]
qubit_hamiltonian = jordan_wigner(molecular_hamiltonian)
evs = eigenspectrum(qubit_hamiltonian)
print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_energies()[0]))
print('MP2 energy of {} Hartree.'.format(molecule.get_energies()[1]))
print('CCSD energy of {} Hartree.'.format(molecule.get_energies()[2]))

Starting Dirac calculation


Creation of the FCIDUMP file

Hartree-Fock energy of -7.8309055873879219 Hartree.
MP2 energy of -7.832575301374425 Hartree.
CCSD energy of -7.832886005802561 Hartree.


The calculation is much faster than if all the spin-orbitals are considered active.

## Special Basis-set

The last option I want to discuss is the special basis set. One can specify a given basis set for a specific element only.
An example is the Helium hydride ion HeH+, where the ```STO-3G``` basis can be considered for the Helium atom and the ```cc-pVDZ``` basis for the hydrogen atom.
To do so, set the basis-set to "special", and defines a special basis as a list of two string, as follows:

In [ ]:
geometry = [('He', (0., 0., 0.)), ('H', (0., 0., bond_length))]
charge = 1
multiplicity = 1
# Due to a known bug (see the end of the tutorial), 
# CCSD does not work for HeH+ without adding some other options manually.
run_ccsd = False
if run_ccsd:
    description = 'R' + str(bond_length) + '_ccsd'
else:
    description = 'R' + str(bond_length) + '_scf'

# necessary keyword to specify that we are using a special basis
basis="special"
# list of two string : [default basis, special basis for a given atomic species]
special_basis=["STO-3G","H BASIS cc-pVDZ"]


molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               special_basis=special_basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               description=description,
                               data_directory=data_directory)

molecule = run_dirac(molecule,
                    point_nucleus=point_nucleus,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    delete_FCIDUMP=delete_FCIDUMP,
                    run_ccsd=run_ccsd)

molecular_hamiltonian = molecule.get_molecular_hamiltonian()[0]
print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_energies()[0]))

Starting Dirac calculation


Creation of the FCIDUMP file

Hartree-Fock energy of -2.8128481516433439 Hartree.


If one wants to make a CCSD calculation on HeH+, it is also possible but an additional option has to be set. This is a known actual issue of Dirac that will be fixed in the next bugfix-patch of Dirac. Meanwhile, it can be fixed by adding an option manually, in order to switch off triple corrections in CCSD.
If one want to add manually more options, see the following subsection.

## Manual options

The Dirac program possesses a lot of different options that can be inserted manually in the script. The ```manual_option``` keyword can be used in this respect.

In [ ]:
run_ccsd = True
if run_ccsd:
    description = 'R' + str(bond_length) + '_ccsd'
else:
    description = 'R' + str(bond_length) + '_scf'

    
basis="STO-3G"
manual_option = """**RELCCSD
*CCENER
.NOSDT""" # or manual_option = "**RELCCSD\n*CCENER\n.NOSDT". This is used to switch off the triples correction.

molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               description=description,
                               data_directory=data_directory)

molecule = run_dirac(molecule,
                    point_nucleus=point_nucleus,
                    manual_option=manual_option,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    delete_FCIDUMP=delete_FCIDUMP,
                    run_ccsd=run_ccsd)

molecular_hamiltonian = molecule.get_molecular_hamiltonian()[0]
qubit_hamiltonian = jordan_wigner(molecular_hamiltonian)
evs = eigenspectrum(qubit_hamiltonian)
print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_energies()[0]))
print('MP2 energy of {} Hartree.'.format(molecule.get_energies()[1]))
print('CCSD energy of {} Hartree.'.format(molecule.get_energies()[2]))
print('Solving the Qubit Hamiltonian (Jordan-Wigner): \n {}'.format(evs))

Starting Dirac calculation


Creation of the FCIDUMP file

Hartree-Fock energy of -2.8107258328141613 Hartree.
MP2 energy of -2.810807100837049 Hartree.
CCSD energy of -2.810780097120332 Hartree.
Solving the Qubit Hamiltonian (Jordan-Wigner): 
 [-3.27136847 -3.27136847 -2.95766144 -2.8107801  -2.39500654 -2.39500654
 -2.39500654 -2.38856976 -2.34400534 -2.34400534 -1.66717093 -1.66717093
 -0.67496425 -0.46136423 -0.46136423  0.52917721]


# Known actual issues 

From the side of Dirac with CCSD:
- HeH+ : Due to the small size of the system, errors when performing CCSD calculation appear. To fix it, one has to switch off the triples correction thanks to the option ```manual_option = "**RELCCSD\n*CCENER\n.NOSDT"```. This will be fixed in the next bugfix-patch of Dirac.


- Open-shell with CCSD: For Be+, HeH, or other open-shell systems, the starting Slater determinant for the CCSD calculation is not well determined. Because of this, CCSD calculation usually get rid of electrons and consider a closed-shell system (like Be2+ instead of Be+, or HeH+ instead of HeH). The repartition of the electrons in the determinant can be specified manually, but no automatic solution has been implemented yet.

From the side of Openfermion:
- Fock space :  The second-quantized Hamiltonian, and so the qubit Hamiltonian, operates in the Fock space. Hence, the ground-state is not always the expected one. This is especially true for ionic species, such as HeH+ for instance. One should add an option to constrain charge (and spin) conservation to get only the states we are interested of. This is yet to be done.

### Open-Shell problems (Be+)

In [ ]:
basis = 'sto-3g'
multiplicity = 2
charge = 1

geometry = [('Be', (0., 0., 0.))]

run_ccsd = 1
description = 'ccsd'

molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               description=description,
                               data_directory=data_directory)

molecule = run_dirac(molecule,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    delete_FCIDUMP=delete_FCIDUMP,
                    run_ccsd=run_ccsd)

molecular_hamiltonian = molecule.get_molecular_hamiltonian()[0]
qubit_hamiltonian = jordan_wigner(molecular_hamiltonian)
evs = eigenspectrum(qubit_hamiltonian)
print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_energies()[0]))
print('MP2 energy of {} Hartree.'.format(molecule.get_energies()[1]))
print('CCSD energy of {} Hartree.'.format(molecule.get_energies()[2]))
print('Solving the Qubit Hamiltonian (Jordan-Wigner): \n {}'.format(evs))

Starting Dirac calculation


Creation of the FCIDUMP file

Hartree-Fock energy of -14.097842446570805 Hartree.
MP2 energy of -13.440020788513420 Hartree.
CCSD energy of -13.439999521071831 Hartree.
Solving the Qubit Hamiltonian (Jordan-Wigner): 
 [-1.44036548e+01 -1.42866219e+01 -1.42866219e+01 ... -1.63415747e+00
 -1.63415747e+00  5.98132655e-15]


Note that the CCSD energy is not the one of Be+, but actually the one of Be(2+) as seen below.

In [ ]:
basis = 'sto-3g'
multiplicity = 1
charge = 2

geometry = [('Be', (0., 0., 0.))]

run_ccsd = 1
description = 'ccsd'

molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               description=description,
                               data_directory=data_directory)

molecule = run_dirac(molecule,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    delete_FCIDUMP=delete_FCIDUMP,
                    run_ccsd=run_ccsd)

molecular_hamiltonian = molecule.get_molecular_hamiltonian()[0]
qubit_hamiltonian = jordan_wigner(molecular_hamiltonian)
evs = eigenspectrum(qubit_hamiltonian)
print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_energies()[0]))
print('MP2 energy of {} Hartree.'.format(molecule.get_energies()[1]))
print('CCSD energy of {} Hartree.'.format(molecule.get_energies()[2]))
print('Solving the Qubit Hamiltonian (Jordan-Wigner): \n {}'.format(evs))

Starting Dirac calculation


Creation of the FCIDUMP file



Now the energy of CCSD is the correct one for Be(2+). However, the ground-state of the qubit Hamiltonian is not the one of Be(2+), because it operates in the whole Fock space.

If one performs a HeH calculation, with ```charge = 0``` and ```multiplicity = 2```, the CCSD calculation will run for HeH+ instead of HeH. Don't forget to set the option ```manual_option = "**RELCCSD\n*CCENER\n.NOSDT"``` to make it work for the HeH molecule.